In [12]:
import pandas as pd
import datetime

submissions = pd.read_csv("Accepted_submissions_info.csv")
sessions = pd.read_csv("session_info.csv")
rooms = pd.read_csv("room_info.csv")
people_info = pd.read_csv("people.csv")

In [13]:
#submissions

def convert_to_first_last(name):
    if "," in name:
        name = name.replace(" ","")
        name = name.replace("\t","")
        last_first_list = name.split(",")
        first = last_first_list[1]
        last = last_first_list[0]
        return first+' '+last
    else:
        return name

people_info["full_name"] = people_info["name"].apply(convert_to_first_last)
list(people_info["name"])
list(people_info["full_name"])

['Meredith Moore', 'John Emmons']

In [14]:
people = {}
for idx in range(len(submissions)):
    curr_sub = submissions.iloc[idx]
    
    authors = curr_sub["authors"].replace("\n","")
    for n in range(10):
        authors = authors.replace(f" ({n})","")
    authors_list = authors.split(";")
    for author_idx in range(len(authors_list)):
        while authors_list[author_idx][0] == " ":
            authors_list[author_idx] = authors_list[author_idx][1:]
        while authors_list[author_idx][-1] == " ":
            authors_list[author_idx] = authors_list[author_idx][:-1]
    #print(authors_list)
    
    organisations = curr_sub["organisations"].replace("\n","")
    for n in range(10):
        organisations = organisations.replace(f"{n}: ","")
    organisations_list = organisations.split(";")
    for organisation_idx in range(len(organisations_list)):
        while organisations_list[organisation_idx][0] == " ":
            organisations_list[organisation_idx] = organisations_list[organisation_idx][1:]
        while organisations_list[organisation_idx][-1] == " ":
            organisations_list[organisation_idx] = organisations_list[organisation_idx][:-1]
    #print(organisations_list)
    
    for author_idx in range(len(authors_list)):
        org = organisations_list[0]
        if len(organisations_list) > 1:
            org = organisations_list[author_idx]
            
        name = authors_list[author_idx]
        first_name = name
        last_name = name
        if "," in name:
            names = name.split(", ")
            first_name = names[1]
            last_name = names[0]
            authors_list[author_idx] = first_name + " " + last_name
        else:
            name_list = name.split(" ")
            first_name = " ".join(name_list[:-1])
            last_name = name_list[-1]
            authors_list[author_idx] = first_name + " " + last_name
            
        people[name] = {"name":first_name+" "+last_name,"first_name":first_name,"last_name":last_name,"organization":org}
        
    
        people_file_data = people_info[people_info["full_name"] == people[name]["name"]]
        if(people[name]["name"] == "Meredith Moore"):
            print(people[name]["name"])
            print(people_info["full_name"])
            print(people_file_data)
            print(len(people_file_data))
        if len(people_file_data) > 0:
            if pd.notna(people_file_data.iloc[0]["affiliation"]):
                people[name]["organization"] = people_file_data.iloc[0]["affiliation"]
            if pd.notna(people_file_data.iloc[0]["image"]):
                people[name]["image"] = people_file_data.iloc[0]["image"]
            if pd.notna(people_file_data.iloc[0]["bio"]):
                people[name]["bio"] = people_file_data.iloc[0]["bio"]
            print(people[name])
        
        
    with open("jekyll-website-test/jekyll-theme-conference/_talks/talk-"+curr_sub["paper ID"]+".md","w") as talkfile:
        talkfile.write("---\n")
        if "'" in curr_sub['title']:
            talkfile.write(f'name: "{curr_sub["title"]}"\n')
        else:
            talkfile.write(f"name: '{curr_sub['title']}'\n")
        talkfile.write("speakers:\n")
        for author_name in authors_list:
            talkfile.write(f"  - {author_name}\n")
        talkfile.write("categories:\n")
        category = curr_sub["contribution type"].replace('"',"")
        talkfile.write(f"  - {category}\n")
        if curr_sub["program_hide"] == True:
            talkfile.write("program_hide: true\n")
        talkfile.write("---\n\n")
        if pd.notna(curr_sub["Abstract"]):
            abstract = curr_sub["Abstract"].replace("\n"," ")
            if "Abstract" == abstract[:8]:
                abstract = abstract[8:]
            talkfile.write(abstract)
        if pd.notna(curr_sub["image"]):
            talkfile.write('\n\n!['+curr_sub['title']+']({{ site.baseurl }}/assets/images/'+curr_sub["image"]+')\n')
        if pd.notna(curr_sub["file"]):
            talkfile.write('\n<p><a href="{{ site.baseurl }}/assets/files/'+curr_sub["file"]+'" class="btn btn-light"><i class="fas fa-file-pdf"></i>&nbsp;Download PDF</a></p>')
        

            
for person in people:
    with open(f'jekyll-website-test/jekyll-theme-conference/_speakers/{people[person]["name"].replace(" ","").replace(".","")}.md',"w") as personfile:
        #print(people[person])
        personfile.write("---\n")
        personfile.write(f"name: {people[person]['name']}\n")
        personfile.write(f"first_name: {people[person]['first_name']}\n")
        personfile.write(f"last_name: {people[person]['last_name']}\n")
        personfile.write(f"affiliation: {people[person]['organization']}\n")
        personfile.write("---\n\n")
        personfile.write(f"{people[person]['organization']}\n")
        if("image" in people[person] and "bio" in people[person]):
            personfile.write('<div style="display: flex; align-items: center; max-width: 1000px; flex-wrap: wrap;">')
            personfile.write('<img src="{{ site.baseurl }}/assets/images/'+people[person]["image"]+'" alt="Headshot" style="max-height:300px; width: auto; display: block; margin-right: 75px;  flex-shrink: 0;">')
            personfile.write('<p style="flex: 1; min-width: 300px;">')
            personfile.write(people[person]["bio"])
            personfile.write('</p>')
            personfile.write('</div>')
        elif "image" in people[person]:
            personfile.write('<img src="{{ site.baseurl }}/assets/images/'+people[person]["image"]+'" alt="Headshot" style="max-height:300px; width: auto; display: block; margin-right: 75px;">')
        elif "bio" in people[person]:
            personfile.write(people[person]["bio"])
        
                    
            
        

Meredith Moore
0    Meredith Moore
1       John Emmons
Name: full_name, dtype: object
              name       affiliation               image  \
0  Moore, Meredith  Drake University  meredithmoore.webp   

                                                 bio       full_name  
0  Dr. Moore is an Assistant Professor of Compute...  Meredith Moore  
1
{'name': 'Meredith Moore', 'first_name': 'Meredith', 'last_name': 'Moore', 'organization': 'Drake University', 'image': 'meredithmoore.webp', 'bio': "Dr. Moore is an Assistant Professor of Computer Science at Drake University. Dr. Moore obtained her B.S. in Computer Science and Neuroscience from Drake University in 2015. She then went on to get her Ph.D. from Arizona State University in Computer Science as a National Science Foundation Graduate Research Fellow.\n\nDr. Moore's research focuses on using machine learning to improve the accessibility of technology for individuals with disabilities. "}
{'name': 'John Emmons', 'first_name': 'John'

In [15]:
#sessions

In [16]:
sessions["day"].unique()

array(['4/4/25', '4/5/25'], dtype=object)

In [17]:
for session_idx in range(len(sessions)):
    curr_session = sessions.iloc[session_idx]
    #print(curr_session)
    if curr_session["type"] != "Break" and curr_session["type"] != "Plenary":
        submissions_in_session = submissions[submissions["session_id"] == curr_session["session_id"]]
        with open(f'jekyll-website-test/jekyll-theme-conference/_sessions/{curr_session["session_title"].replace(" ","").replace(".","")}.md',"w") as sessionfile:
            if "'" in curr_session["session_title"]:
                sessionfile.write(f'---\nname: "{curr_session["session_title"]}"\n---\n')
            else:
                sessionfile.write(f"---\nname: '{curr_session['session_title']}'\n---\n")
            

In [18]:
def parse_date(day_str):
    # Try ISO format first
    try:
        return datetime.datetime.fromisoformat(day_str)
    except ValueError:
        pass
    # Try Excel's M/D/YY or MM/DD/YY
    try:
        return datetime.datetime.strptime(day_str, "%m/%d/%y")
    except ValueError:
        pass
    # Try D/M/YY if you're dealing with international formats
    try:
        return datetime.datetime.strptime(day_str, "%d/%m/%y")
    except ValueError:
        pass
    
    raise ValueError(f"Unrecognized date format: {day_str}")
   
used_room_order = []  # To store the order of rooms as they first appear

with open("jekyll-website-test/jekyll-theme-conference/_data/program.yml","w") as progfile:
    progfile.write("days:\n")
    
    for day in sessions["day"].unique():
        
        sessions_on_day = sessions[sessions["day"]==day]
        
        #dt = datetime.datetime.fromisoformat(day)
        dt = parse_date(day)
        weekday = dt.strftime("%A")
        weekday_abbr = dt.strftime("%a")
        
        progfile.write(f"- name: {weekday}\n")
        progfile.write(f"  abbr: {weekday_abbr}\n")
        #progfile.write(f"  date: {day}\n")
        progfile.write(f"  date: {dt.strftime('%Y-%m-%d')}\n")
        progfile.write("  rooms:\n")
        
        # Rooms used today
        rooms_today = sessions_on_day["room"].dropna().unique().tolist()

        # 1. Rooms seen before that are used today (preserve their original order)
        ordered_today = [room for room in used_room_order if room in rooms_today]

        # 2. New rooms not seen before
        new_rooms = [room for room in rooms_today if room not in used_room_order]
        ordered_today += new_rooms

        # 3. Update the global room order tracker
        for room in new_rooms:
            used_room_order.append(room)

        
        for room in ordered_today:   
            
            
            if pd.notna(room):
            
                progfile.write(f"  - name: {room}\n")

                sessions_in_room = sessions_on_day[sessions_on_day["room"] == room]

                progfile.write("    sessions:\n")

                for session_id in sessions_in_room["session_id"].unique():

                    curr_session = sessions_in_room[sessions_in_room["session_id"] == session_id].iloc[0]

                    progfile.write(f"    - name: {curr_session.session_title}\n")

                    progfile.write(f"      category: '{curr_session.type}'\n")
                    if curr_session.plenary == True:
                        progfile.write(f"      plenary: true\n")
                    else:
                        progfile.write(f"      plenary: false\n")
                    if pd.notna(curr_session.description):
                        progfile.write(f"      description: '{curr_session.description}'\n")
                    progfile.write(f"      time_start: '{curr_session.start_time}'\n")
                    progfile.write(f"      time_end: '{curr_session.end_time}'\n")
                    if pd.notna(curr_session["moderator"]):
                        progfile.write(f"      moderator: '{curr_session.moderator}'\n")

                    talks = submissions[submissions["session_id"] == session_id]

                    if len(talks) > 0:
                        progfile.write(f"      talks: \n")

                        for talk_idx in range(len(talks)):
                            curr_talk = talks.iloc[talk_idx]
                            if "'" in curr_talk['title']:
                                progfile.write(f'      - name: "{curr_talk["title"]}"\n')
                            else:
                                progfile.write(f"      - name: '{curr_talk['title']}'\n")






In [19]:
rooms

,room,room_img1,room_img2,directions
0,C-S 301,C-S301Map.png,C-SBuilding.jpg,"On the 3rd floor of Collier-Scripps Hall, from..."
1,C-S 207,C-S207Map.png,C-SBuilding.jpg,"On the 2nd floor of Collier-Scripps Hall, from..."
2,C-S 235,C-S235Map.png,C-SBuilding.jpg,"On the 2nd floor of Collier-Scripps Hall, from..."
3,C-S 308 and Halls,C-S3rdFloorMap.png,C-SBuilding.jpg,"You can find refreshments, vendor tables, and ..."
4,Shivers,shivers.jpeg,NaN,Enter the Shivers Basketball Practice Facility...
5,Des Moines Restaurants,NaN,NaN,\n<h3>Adjacent to Campus</h3>\n<ul>\n<li><a hr...
6,Upper Carnegie,carnegie.jpg,NaN,Enter through the main entrance on the north s...
7,Olin 101,Olin.jpg,NaN,Olin Hall is the building just north of Collie...


In [20]:
for room_idx in range(len(rooms)):
    room = rooms.iloc[room_idx]
    display(room)
    with open("jekyll-website-test/jekyll-theme-conference/_rooms/"+room["room"].replace(" ","")+".md","w") as roomfile:
        roomfile.write(f"---\nname: {room['room']}\n---\n")
        if pd.notna(room["directions"]):
            roomfile.write("<br/><br/>")
            roomfile.write(f'{room["directions"]}')
        if pd.notna(room["room_img1"]):
            roomfile.write("<br/><br/>")
            roomfile.write('![room image]({{ site.baseurl }}'+f'/assets/images/{room["room_img1"]})'+'{: style="max-height:400px; width: auto; display: block; margin: 0 auto;" }')
        if pd.notna(room["room_img2"]):
            roomfile.write("<br/><br/>")
            roomfile.write('![room image]({{ site.baseurl }}'+f'/assets/images/{room["room_img2"]})'+'{: style="max-height:400px; width: auto; display: block; margin: 0 auto;" }')

        
                

room                                                    C-S 301
room_img1                                         C-S301Map.png
room_img2                                       C-SBuilding.jpg
directions    On the 3rd floor of Collier-Scripps Hall, from...
Name: 0, dtype: object

room                                                    C-S 207
room_img1                                         C-S207Map.png
room_img2                                       C-SBuilding.jpg
directions    On the 2nd floor of Collier-Scripps Hall, from...
Name: 1, dtype: object

room                                                    C-S 235
room_img1                                         C-S235Map.png
room_img2                                       C-SBuilding.jpg
directions    On the 2nd floor of Collier-Scripps Hall, from...
Name: 2, dtype: object

room                                          C-S 308 and Halls
room_img1                                    C-S3rdFloorMap.png
room_img2                                       C-SBuilding.jpg
directions    You can find refreshments, vendor tables, and ...
Name: 3, dtype: object

room                                                    Shivers
room_img1                                          shivers.jpeg
room_img2                                                   NaN
directions    Enter the Shivers Basketball Practice Facility...
Name: 4, dtype: object

room                                     Des Moines Restaurants
room_img1                                                   NaN
room_img2                                                   NaN
directions    \n<h3>Adjacent to Campus</h3>\n<ul>\n<li><a hr...
Name: 5, dtype: object

room                                             Upper Carnegie
room_img1                                          carnegie.jpg
room_img2                                                   NaN
directions    Enter through the main entrance on the north s...
Name: 6, dtype: object

room                                                   Olin 101
room_img1                                              Olin.jpg
room_img2                                                   NaN
directions    Olin Hall is the building just north of Collie...
Name: 7, dtype: object